# Scrapping con BS4

En este notebook realizo Scrapping basado en el urso de Web Scrapping de Python, utilizaré el periodico Colombiano EL TIEMPO para hacer mi propia version de Scrapping

#### Solicitudes de páginas web
Para realizar las peticiones a páginas web haremos uso de la librería request de Python

In [1]:
import requests

In [2]:
url = 'https://www.eltiempo.com/'

In [3]:
response = requests.get(url)

In [4]:
response.status_code

200

##### Status 200 code = Request ejecutado con éxito

In [5]:
response.text
#Imprimimos como texto el response obtenido

'<!DOCTYPE html>\n<html>\n<head>\n<script async type=\'text/javascript\'>var _sf_startpt=(new Date()).getTime()</script>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<script type="text/javascript">\nwindow.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o||e)},o,o.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({1:[function(t,e,n){function r(t){try{s.console&&console.log(t)}catch(e){}}var o,i=t("ee"),a=t(23),s={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.console=!0,o.indexOf("dev")!==-1&&(s.dev=!0),o.indexOf("nr_dev")!==-1&&(s.nrDev=!0))}catch(c){}s.nrDev&&i.on("internal-error",function(t){r(t.stack)}),s.dev&&i.on("fn-err",function(t,e,n){r(n.stack)}),s.dev&&(r("NR AGENT IN DEVELOPMENT MODE"),r("flags: "+a(s,function(t,e

In [6]:
response.content
#Para casos donde obtenemos archivos multimedia

b'<!DOCTYPE html>\n<html>\n<head>\n<script async type=\'text/javascript\'>var _sf_startpt=(new Date()).getTime()</script>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<script type="text/javascript">\nwindow.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o||e)},o,o.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({1:[function(t,e,n){function r(t){try{s.console&&console.log(t)}catch(e){}}var o,i=t("ee"),a=t(23),s={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.console=!0,o.indexOf("dev")!==-1&&(s.dev=!0),o.indexOf("nr_dev")!==-1&&(s.nrDev=!0))}catch(c){}s.nrDev&&i.on("internal-error",function(t){r(t.stack)}),s.dev&&i.on("fn-err",function(t,e,n){r(n.stack)}),s.dev&&(r("NR AGENT IN DEVELOPMENT MODE"),r("flags: "+a(s,function(t,

In [7]:
response.headers
#Encabezado de la página web

{'Server': 'nginx', 'Content-Type': 'text/html; charset=UTF-8', 'X-Server': 'fet04', 'Strict-Transport-Security': 'max-age=15768000; includeSubDomains, max-age=157680000', 'x-url': '/', 'x-frame-options': 'sameorigin', 'X-Cacheable': 'YES', 'X-Varnish': '967674247 1035087965', 'X-sc': 'fetx3', 'X-CDN': 'Imperva', 'Content-Encoding': 'gzip', 'X-Iinfo': '9-66702214-66762705 SNYy RT(1628190891891 424904) q(0 1 1 -1) r(1 1) U5', 'X-Akamai-Transformed': '9 - 0 pmb=mRUM,1', 'Cache-Control': 'max-age=120', 'Expires': 'Thu, 05 Aug 2021 19:29:02 GMT', 'Date': 'Thu, 05 Aug 2021 19:27:02 GMT', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive, Transfer-Encoding', 'Vary': 'Accept-Encoding', 'Server-Timing': 'cdn-cache; desc=HIT, edge; dur=1', 'version': 'desktop'}

In [8]:
#Encabezado de NUESTRA solicitud:
response.request.headers

{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

###### Observemos como el servidor encuentra que se realizo una solicitud usando el software de python, este user-agent en algunos casos debe ser cambiado para acceder a ciertas wb

In [9]:
#Repaso de nuestro método GET:
response.request.method

'GET'

In [10]:
#Confirmamos la URL solicitada con:
response.request.url

'https://www.eltiempo.com/'

### Parseando HTML con BeatifulSoup

In [11]:
from bs4 import BeautifulSoup

In [12]:
# Parseamos el codigo HTML
s = BeautifulSoup(response.text, 'lxml')

In [13]:
type(s)

bs4.BeautifulSoup

In [14]:
#Mejoramos la presentación del texto html con prettify()
print(s.prettify())

<!DOCTYPE html>
<html>
 <head>
  <script async="" type="text/javascript">
   var _sf_startpt=(new Date()).getTime()
  </script>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <script type="text/javascript">
   window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o||e)},o,o.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({1:[function(t,e,n){function r(t){try{s.console&&console.log(t)}catch(e){}}var o,i=t("ee"),a=t(23),s={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.console=!0,o.indexOf("dev")!==-1&&(s.dev=!0),o.indexOf("nr_dev")!==-1&&(s.nrDev=!0))}catch(c){}s.nrDev&&i.on("internal-error",function(t){r(t.stack)}),s.dev&&i.on("fn-err",function(t,e,n){r(n.stack)}),s.dev&&(r("NR AGENT IN DEVELOPMENT MODE"),r("flags: "+a(s,fu

### Revisando estructura de la página EL TIEMPO.

Como podemos observar, la barra de navegacion de secciones está en el div identificado con class = 'default-menu'

In [15]:
# Obtener encabezado de la página:
#s.find('ul') #De esta forma solo retorna el primer ul
s.find('ul', attrs={'class' : 'default-menu'})

<ul class="default-menu">
<li class="opinion first" itemprop="name">
<a href="/opinion" itemprop="url">Opinión</a>
</li>
<li class="colombia" itemprop="name">
<a href="/colombia" itemprop="url">Colombia</a>
</li>
<li class="bogota" itemprop="name">
<a href="/bogota" itemprop="url">Bogotá</a>
</li>
<li class="mundo" itemprop="name">
<a href="/mundo" itemprop="url">Internacional</a>
</li>
<li class="politica" itemprop="name">
<a href="/politica" itemprop="url">Política</a>
</li>
<li class="justicia" itemprop="name">
<a href="/justicia" itemprop="url">Justicia</a>
</li>
<li class="economia" itemprop="name">
<a href="/economia" itemprop="url">Economía</a>
</li>
<li class="deportes" itemprop="name">
<a href="/deportes" itemprop="url">Deportes</a>
</li>
<li class="cultura" itemprop="name">
<a href="/cultura" itemprop="url">Cultura</a>
</li>
<li class="tecnosfera" itemprop="name">
<a href="/tecnosfera" itemprop="url">Tecnología</a>
</li>
<li class="vida" itemprop="name">
<a href="/vida" itemp

Sin embargo, realmente lo que nos importa es el elemento li (list item) contenido en el tag ul, para ello usamos:

In [16]:
secciones = s.find('ul', attrs={'class' : 'default-menu'}).find_all('li')
secciones

[<li class="opinion first" itemprop="name">
 <a href="/opinion" itemprop="url">Opinión</a>
 </li>,
 <li class="colombia" itemprop="name">
 <a href="/colombia" itemprop="url">Colombia</a>
 </li>,
 <li class="bogota" itemprop="name">
 <a href="/bogota" itemprop="url">Bogotá</a>
 </li>,
 <li class="mundo" itemprop="name">
 <a href="/mundo" itemprop="url">Internacional</a>
 </li>,
 <li class="politica" itemprop="name">
 <a href="/politica" itemprop="url">Política</a>
 </li>,
 <li class="justicia" itemprop="name">
 <a href="/justicia" itemprop="url">Justicia</a>
 </li>,
 <li class="economia" itemprop="name">
 <a href="/economia" itemprop="url">Economía</a>
 </li>,
 <li class="deportes" itemprop="name">
 <a href="/deportes" itemprop="url">Deportes</a>
 </li>,
 <li class="cultura" itemprop="name">
 <a href="/cultura" itemprop="url">Cultura</a>
 </li>,
 <li class="tecnosfera" itemprop="name">
 <a href="/tecnosfera" itemprop="url">Tecnología</a>
 </li>,
 <li class="vida" itemprop="name">
 <a hr

### Hora de extraer la información contenida en los tag! , nuestro objetivo los href

In [17]:
seccion = secciones[0]

In [18]:
seccion

<li class="opinion first" itemprop="name">
<a href="/opinion" itemprop="url">Opinión</a>
</li>

In [19]:
#Entrar al tag a

In [20]:
#Método 1
print(seccion.find('a'))

#Método 2
print(seccion.a)

<a href="/opinion" itemprop="url">Opinión</a>
<a href="/opinion" itemprop="url">Opinión</a>


In [21]:
#Objetivo: Obtener href y texto
print("Texto: ", seccion.a.get_text())
print("URL: ", seccion.a.get('href'))

Texto:  Opinión
URL:  /opinion


##### Este es un caso particular, podemos obtener exactamente el link parseando con la URL

In [22]:
print("URL: ", url+seccion.a.get('href')[1:])

URL:  https://www.eltiempo.com/opinion


### Obtener todos los links desde secciones

Vamos a realizar la obtención de todos los links mediante un list_comprehension

Adicional, recordando la particularidad vista anteriormente, creo una función para añadir la URL al href

In [23]:
def add_url(url: str,href: str)-> str:
    return url+href[1:]

In [24]:
link_secciones = [add_url(url, seccion.a.get('href')) for seccion in secciones]
link_secciones

['https://www.eltiempo.com/opinion',
 'https://www.eltiempo.com/colombia',
 'https://www.eltiempo.com/bogota',
 'https://www.eltiempo.com/mundo',
 'https://www.eltiempo.com/politica',
 'https://www.eltiempo.com/justicia',
 'https://www.eltiempo.com/economia',
 'https://www.eltiempo.com/deportes',
 'https://www.eltiempo.com/cultura',
 'https://www.eltiempo.com/tecnosfera',
 'https://www.eltiempo.com/vida',
 'https://www.eltiempo.com/salud',
 'https://www.eltiempo.com/unidad-investigativa',
 'https://www.eltiempo.com/eltiempoplay']

In [25]:
texto_secciones = [seccion.a.get_text() for seccion in secciones]
texto_secciones

['Opinión',
 'Colombia',
 'Bogotá',
 'Internacional',
 'Política',
 'Justicia',
 'Economía',
 'Deportes',
 'Cultura',
 'Tecnología',
 'Vida',
 'Salud',
 'Unidad Investigativa',
 'El Tiempo Play']

### Mostrando nuestro Scrappeo :D

In [26]:
for idx,texto in enumerate(texto_secciones):
    print(f"Sección N° {idx}. {texto:>3}, accede a ella mediante el link: {link_secciones[idx]:>}\n")

Sección N° 0. Opinión, accede a ella mediante el link: https://www.eltiempo.com/opinion

Sección N° 1. Colombia, accede a ella mediante el link: https://www.eltiempo.com/colombia

Sección N° 2. Bogotá, accede a ella mediante el link: https://www.eltiempo.com/bogota

Sección N° 3. Internacional, accede a ella mediante el link: https://www.eltiempo.com/mundo

Sección N° 4. Política, accede a ella mediante el link: https://www.eltiempo.com/politica

Sección N° 5. Justicia, accede a ella mediante el link: https://www.eltiempo.com/justicia

Sección N° 6. Economía, accede a ella mediante el link: https://www.eltiempo.com/economia

Sección N° 7. Deportes, accede a ella mediante el link: https://www.eltiempo.com/deportes

Sección N° 8. Cultura, accede a ella mediante el link: https://www.eltiempo.com/cultura

Sección N° 9. Tecnología, accede a ella mediante el link: https://www.eltiempo.com/tecnosfera

Sección N° 10. Vida, accede a ella mediante el link: https://www.eltiempo.com/vida

Sección 

## Ya podemos Scrappear las secciones a partir de los links obtenidos

In [27]:
sec = requests.get(link_secciones[0]) #Obtenemos la pagina de Opinión

In [28]:
sec.status_code #Verificamos que todo está correcto

200

In [29]:
s_seccion = BeautifulSoup(sec.text, 'lxml')

In [30]:
print(s_seccion.prettify())

<!DOCTYPE html>
<html>
 <head>
  <script async="" type="text/javascript">
   var _sf_startpt=(new Date()).getTime()
  </script>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <script type="text/javascript">
   window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o||e)},o,o.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({1:[function(t,e,n){function r(t){try{s.console&&console.log(t)}catch(e){}}var o,i=t("ee"),a=t(23),s={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.console=!0,o.indexOf("dev")!==-1&&(s.dev=!0),o.indexOf("nr_dev")!==-1&&(s.nrDev=!0))}catch(c){}s.nrDev&&i.on("internal-error",function(t){r(t.stack)}),s.dev&&i.on("fn-err",function(t,e,n){r(n.stack)}),s.dev&&(r("NR AGENT IN DEVELOPMENT MODE"),r("flags: "+a(s,fu

In [31]:
#columnas = s_seccion.find('div', attrs={'class': 'columnistas-container'}).find('div', attrs = {'class':'opinion-author-articles'})
columnas = s_seccion.find('div', attrs={'id' : 'ls-184-b14'}).find_all('article')
columnas

[<article class="opinion-author-module columnist-module" id="m6850-6849-6851" itemscope="" itemtype="http://schema.org/NewsArticle"><div class="recurso" itemtype="https://schema.org/NewsArticle">
 <figure class="author-image-container"><a class="image page-link" href="/opinion/columnistas/juan-esteban-constain/senoras-y-senores-columna-de-juan-esteban-constain-608393" id="m6855-6854-6856" itemprop="image" itemscope="" itemtype="https://schema.org/ImageObject"><picture></picture><meta content="80" itemprop="width"/><meta content="80" itemprop="height"/><meta content="https://www.eltiempo.com/files/image_80_80/uploads/2021/04/07/606e452229735.jpeg" itemprop="url"/></a></figure></div>
 <div class="content-container">
 <div class="author-container">
 <a class="author-name page-link" href="/opinion/columnistas/juan-esteban-constain" id="m6862-1-6863">Juan Esteban Constaín</a></div>
 <h3 class="title-container" itemprop="Headline">
 <a class="title page-link" href="/opinion/columnistas/juan-

In [32]:
scrapping_opinion_columns = {
    'titulos' : [columna.find('a', attrs = {'class': 'title page-link'}).get_text() for columna in columnas],
    'autores' : [columna.find('a', attrs = {'class': 'author-name page-link'}).get_text() for columna in columnas],
    'links' : [add_url(url,columna.find('a', attrs = {'class': 'title page-link'}).get('href')) for columna in columnas]
}

In [33]:
print("*"*100,"\nSíntesis de Scrapping realizado a blogs en la sección Opinión\n","*"*100,"\n\n")

for idx,titulo in enumerate(scrapping_opinion_columns['titulos']):
    print(f"Columna de opinión N°{idx+1}. {titulo}.\nAutor: {scrapping_opinion_columns['autores'][idx]}.\nLink: {scrapping_opinion_columns['links'][idx]}\n")

**************************************************************************************************** 
Síntesis de Scrapping realizado a blogs en la sección Opinión
 **************************************************************************************************** 


Columna de opinión N°1. Señoras y señores.
Autor: Juan Esteban Constaín.
Link: https://www.eltiempo.com/opinion/columnistas/juan-esteban-constain/senoras-y-senores-columna-de-juan-esteban-constain-608393

Columna de opinión N°2. ¿Estado social de derecho?.
Autor: Pedro Medellin.
Link: https://www.eltiempo.com/opinion/columnistas/pedro-medellin/estado-social-de-derecho-columna-de-pedro-medellin-608391

Columna de opinión N°3. Preguntas para la Conferencia Episcopal.
Autor: Claudia Palacios.
Link: https://www.eltiempo.com/opinion/columnistas/claudia-palacios/preguntas-para-la-conferencia-episcopal-columna-de-claudia-palacios-608389

Columna de opinión N°4. La lágrima fácil.
Autor: Sergio Ramírez.
Link: https://www.eltiempo.

## Síntesis de Scrapping de la sección de opinión (COLUMNAS)

In [34]:
blogs = s_seccion.find('div', attrs={'class': 'opinion-board opinion-blogs-board'}).find_all('article')
blogs
#Blogs capturados

[<article class="opinion-author-module blog-module" id="p73-m10960-10959-10961" itemscope="" itemtype="https://schema.org/NewsArticle"><div class="recurso" itemtype="https://schema.org/NewsArticle">
 <figure class="author-image-container"><a class="image page-link" href="https://blogs.eltiempo.com/mil-palabras/2021/07/27/podcast-felicidad-organizacional-125/" id="m10965-10964-10966" itemprop="image" itemscope="" itemtype="https://schema.org/ImageObject"><picture></picture><meta content="80" itemprop="width"/><meta content="80" itemprop="height"/><meta content="https://www.eltiempo.com/files/image_80_80/uploads/2017/09/22/59c55aea9f6bd.jpeg" itemprop="url"/></a></figure></div>
 <div class="content-container">
 <div class="category-container">
 <a class="category page-link undefined" href="https://blogs.eltiempo.com/mil-palabras/2021/07/27/podcast-felicidad-organizacional-125/" id="m10972-1-10973" title="Mil Palabras">Mil Palabras</a></div>
 <div class="author-container">
 <span class="a

In [35]:
scrapping_blogs = {
    'titulos' : [blog.find('h3', attrs = {'class': 'title-container'}).get_text()[1:] for blog in blogs],
    'autores' : [blog.find('span', attrs = {'class' : 'author-name'}).get_text() for blog in blogs],
    'links' : [blog.find('a', attrs = {'class' : 'title page-link'}).get('href') for blog in blogs]
}

## Síntesis de Scrapping de la sección de opinión (BLOGS)

In [36]:
print("*"*100,"\nSíntesis de Scrapping realizado a blogs en la sección Opinión\n","*"*100,"\n\n")

for idx,titulo in enumerate(scrapping_blogs['titulos']):
    print(f"Columna de opinión N°{idx+1}. {titulo}.\nAutor: {scrapping_blogs['autores'][idx]}.\nLink: {scrapping_blogs['links'][idx]}\n")

**************************************************************************************************** 
Síntesis de Scrapping realizado a blogs en la sección Opinión
 **************************************************************************************************** 


Columna de opinión N°1. [PODCAST] felicidad Organizacional #125.
Autor: Santiago Ríos.
Link: https://blogs.eltiempo.com/mil-palabras/2021/07/27/podcast-felicidad-organizacional-125/

Columna de opinión N°2. La bandera es una sola y es de todos los colombianos.
Autor: Fabio Cuello.
Link: https://blogs.eltiempo.com/el-observador-pensante/2021/07/27/la-bandera-es-una-sola-y-es-de-todos-los-colombianos/

Columna de opinión N°3. La inevitable victoria de los talibanes en Afganistán.
Autor: Mauricio Jaramillo Jassir.
Link: https://blogs.eltiempo.com/geopolitica-poder-y-democracia/2021/07/27/la-inevitable-victoria-de-los-talibanes-en-afganistan/

Columna de opinión N°4. Marketing de las abuelas.
Autor: Fabián Salazar.
Link: http